In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tkinter import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


df = pd.DataFrame(pd.read_csv('hour1.csv'))

In [11]:
df['dateday'] = pd.to_datetime(df['dateday'], format='%m/%d/%Y')
day = df.insert(loc=5, column='day', value=df['dateday'].dt.day)

In [12]:
df1 = df.copy()  # make a copy of the original dataframe

df1.loc[df1['holiday'] == 0, 'holiday'] = 'No'
df1.loc[df1['holiday'] == 1, 'holiday'] = 'Yes'

df1.loc[df1['workingday'] == 0, 'workingday'] = 'No'
df1.loc[df1['workingday'] == 1, 'workingday'] = 'Yes'

df1.loc[df1['season'] == 1, 'season'] = 'Winter'
df1.loc[df1['season'] == 2, 'season'] = 'Spring'
df1.loc[df1['season'] == 3, 'season'] = 'Summer'
df1.loc[df1['season'] == 4, 'season'] = 'Fall'

df1.loc[df1['weather'] == 1, 'weather'] = '1_Clear'
df1.loc[df1['weather'] == 2, 'weather'] = '2_Cloud'
df1.loc[df1['weather'] == 3, 'weather'] = '3_Light'
df1.loc[df1['weather'] == 4, 'weather'] = '4_Heavy'

In [16]:
display(df.head(1))
display(df1.head(1))

,instant,dateday,season,year,month,day,hour,holiday,weekday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,1,2011-07-03,3,0,7,3,17,0,0,0,3,0.8,0.7424,0.49,0.8507,181,177,358


,instant,dateday,season,year,month,day,hour,holiday,weekday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,1,2011-07-03,Summer,0,7,3,17,No,0,No,3_Light,0.8,0.7424,0.49,0.8507,181,177,358


In [36]:
X = df.drop(columns=['dateday', 'count', 'year'])
y = df['count']


In [37]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,  random_state=10)

In [38]:
model_pipeline = []
model_pipeline.append(LogisticRegression(solver='liblinear'))
model_pipeline.append(RandomForestClassifier())
model_pipeline.append(SVC())
model_pipeline.append(KNeighborsClassifier())
model_pipeline.append(DecisionTreeClassifier())
model_pipeline.append(GaussianNB())

In [39]:
model_list = ['LogisticRegression', 'Random Forest', 'SVM', 'KNN', 'Decision Tree', 'Naive Bayes']
acc_list = []
auc_list = []
cm_list = []
ck_list = []

for model in model_pipeline:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc_list.append(round(metrics.accuracy_score(y_test, y_pred),2))
    fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
    auc_list.append(round(metrics.auc(fpr, tpr), 2))
    cm_list.append(confusion_matrix(y_test, y_pred))
    ck_list.append(y_pred)